<a href="https://colab.research.google.com/github/CharlottePrimiceri/VP_Project/blob/main/Segmented_video_Mask_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Segmented video generation applying the pre-trained Mask-RCNN.**

THIS SCRIPT TAKES A VIDEO IN INPUT FROM CITYSCAPES AND ELABORATES ALL ITS FRAMES WITH OUR PYTORCH UNET RIVISITED. THEN, THE OPTICAL FLOW VALUE IS COMPUTED ON THE SEGMNETED IMAGES, WITH THE AIM OF ESTIMATING THE SPEED OF OBJECTS IN THE VIDEO.

References:

- Mask-RCNN:
https://github.com/facebookresearch/detectron2.git@5aeb252b194b93dc2879b4ac34bc51a31b5aee13'
- Video taken from: https://www.cityscapes-dataset.com/

## Install detectron2, import libraries, connect to google drive, set device.

In [ ]:
!pip install torch --quiet

In [ ]:
!pip install torchvision --quiet

In [ ]:
! pip install scikit-image --quiet

In [ ]:
! pip install opencv-python --quiet

In [ ]:
! pip install Pillow

In [ ]:
import os

from PIL import Image
import cv2
import matplotlib.pyplot as plt

import numpy as np
import torch

#######
import scipy
import skimage
import PIL
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from google.colab.patches import cv2_imshow
########

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Mounted at /content/drive


In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git@5aeb252b194b93dc2879b4ac34bc51a31b5aee13' --quiet

  Preparing metadata (setup.py) ... done


In [ ]:
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

## Detectron2 configuration

In [ ]:
cfg = get_cfg()
cfg.MODEL.DEVICE = "cpu"
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

predictor = DefaultPredictor(cfg)

In [ ]:
video_path = "/content/drive/MyDrive/VPPROJECT/EUREKA/video_making/demoVideo/stuttgart_02/" # immagini non segmentate
segmented_video_path = "/content/drive/MyDrive/VPPROJECT/EUREKA/video_making/demoVideo/segmented_video/" # immagini segmentate
optical_flow_path = "/content/drive/MyDrive/VPPROJECT/EUREKA/video_making/demoVideo/opt_flow_for_video/" # immagini su cui è stato applicato opt_flow
built_video_path = "/content/drive/MyDrive/VPPROJECT/EUREKA/video_making/demoVideo/built_video/"



def generate_video(img_folder, video_name, out_path):

    images = [img for img in os.listdir(img_folder)]
    frame = cv2.imread(os.path.join(img_folder, images[0]))
    height, width, layers = frame.shape

    os.chdir(out_path)

    video = cv2.VideoWriter(video_name, 0, 1, (width, height))   # try 2,3,4 al posto di 1 per velocizzare

    for image in images:
        video.write(cv2.imread(os.path.join(img_folder, image)))

    cv2.destroyAllWindows()
    video.release()


i = 0
for image in os.listdir(video_path):
    i = i+1
    if i % 5 == 0:
        image_path = os.path.join(video_path, image)
        im = cv2.imread(image_path)

        # here we segment the image with the pre-trained Mask-RCNN
        pred = predictor(im)
        v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
        out = v.draw_instance_predictions(pred["instances"].to("cpu"))
        path = os.path.join(segmented_video_path, image)
        cv2.imwrite(path, out.get_image()[:, :, ::-1])

        # here we compute the optical flow ############# TO DOOOO

# Build video from folder segmented_video
generate_video(segmented_video_path, "segmented_video_1.avi", built_video_path)
